<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  PCA para estudiar la dimensionalidad de la calidad de vida en las ciudades 

En este caso-taller vamos a aplicar PCA en el estudio de calidad de vida en diferentes ciudades. Para ello, haremos uso de los datos que surgen del Almanaque Calificado de Lugares. En este Almanaque Boyer y Savageau calificaron 329 ciudades de acuerdo con los siguientes nueve criterios:

   - Clima y Terreno
   - Alojamiento
   - Cuidado de la salud y el medio ambiente
   - Crimen
   - Transporte
   - Educación
   - Artes
   - Recreación
   - Economía 

Con tantas variables disponibles, la matriz de dispersión puede resultar muy grande para estudiarla e interpretarla adecuadamente. Habría demasiadas correlaciones bivariadas entre las variables a considerar. Para interpretar los datos de una forma más significativa es necesario reducir el número de variables a unas pocas dimensiones, y para ello en este ejercicio recurriremos al análisis de componentes principales. 
 

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo junto al archivo de soporte.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en la misma ruta de acceso del cuaderno, por ejemplo: `data/archivo_de_soporte.csv`.

## Desarrollo


#### Config

##### Carga de Bibliotecas

In [21]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

# python
import os
import sys
import csv
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import session_info

from pathlib import Path

from numpy.linalg import matrix_power

# rutas absolutas
here: Path = Path.cwd().absolute()
data: Path = here / 'data'
input: str = str(data / 'lugares.csv')
output: str = str(data / 'archivo_de_soporte.csv')

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# paquete pca
from pca import pca

# setup
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', None)
# decimals
np.set_printoptions(precision=6)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Información de Sesión

In [17]:
session_info.show(html=False)

-----
matplotlib          3.10.0
numpy               1.26.4
pandas              2.3.1
seaborn             0.13.2
session_info        v1.0.1
-----
IPython             8.37.0
jupyter_client      8.6.3
jupyter_core        5.8.1
jupyterlab          4.4.5
notebook            7.4.5
-----
Python 3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]
macOS-10.15.7-x86_64-i386-64bit
-----
Session information updated at 2025-08-08 21:55


### 1. Carga de datos

En la carpeta `data` se encuentra el archivo `lugares.csv` cargue estos datos en su *cuaderno*.

In [8]:
# Utilice este espacio para escribir el código.
df = pd.read_csv(input, sep=',', encoding='utf-8')
df.set_index('Ciudad', inplace=True)
df.head(5)

,Clima y Terreno,Alojamiento,Cuidado de la salud y el medio ambiente,Crimen,Transporte,Educación,Artes,Recreación,Economía
Ciudad,,,,,,,,,
"Abilene,TX",521,6200,237,923,4031,2757,996,1405,7633
"Akron,OH",575,8138,1656,886,4883,2438,5564,2632,4350
"Albany,GA",468,7339,618,970,2531,2560,237,859,5250
"Albany-Schenectady-Troy,NY",476,7908,1431,610,6883,3399,4655,1617,5864
"Albuquerque,NM",659,8393,1853,1483,6558,3026,4496,2612,5727


### 2.  Análisis descriptivo de las variables. 

Para el análisis descriptivo haga uso de estadísticas descriptivas y matrices de correlación. Indique además cuantas gráficas de dispersión bivariadas debería examinar con este número de variables. 

Explique porqué es importante conocer las estadísticas descriptivas y como contribuyen al ejercicio posterior de Análisis de Componentes Principales.
 
Tenga en cuenta que para las variables, a excepción de la alojamiento y crimen, cuanto mayor sea el puntaje, mejor. Por el contrario; para alojamiento y crimen, mientras más bajo sea el puntaje, mejor. También, mientras algunas comunidades podrían calificar mejor en las artes, otras podrían calificar mejor en otras áreas como tener una tasa de criminalidad más baja y buenas oportunidades educativas.

In [23]:
# Utilice este espacio para escribir el código.
# Estadísticas descriptivas
descriptive_stats = df.describe()
descriptive_stats.T

,count,mean,std,min,25%,50%,75%,max
Clima y Terreno,329.0,538.732523,120.808259,105.0,480.0,542.0,592.0,910.0
Alojamiento,329.0,8346.559271,2385.262622,5159.0,6760.0,7877.0,9015.0,23640.0
Cuidado de la salud y el medio ambiente,329.0,1185.738602,1003.002036,43.0,583.0,833.0,1445.0,7850.0
Crimen,329.0,961.054711,357.154186,308.0,707.0,947.0,1156.0,2498.0
Transporte,329.0,4210.082067,1451.179240,1145.0,3141.0,4080.0,5205.0,8625.0
Educación,329.0,2814.887538,320.792953,1701.0,2619.0,2794.0,3012.0,3781.0
Artes,329.0,3150.884498,4642.283738,52.0,778.0,1871.0,3844.0,56745.0
Recreación,329.0,1845.957447,807.888171,300.0,1316.0,1670.0,2176.0,4800.0
Economía,329.0,5525.364742,1084.468523,3045.0,4842.0,5384.0,6113.0,9980.0


In [24]:
# Matriz de correlación
correlation_matrix = df.corr()
correlation_matrix

,Clima y Terreno,Alojamiento,Cuidado de la salud y el medio ambiente,Crimen,Transporte,Educación,Artes,Recreación,Economía
Clima y Terreno,1.000000,0.386291,0.213303,0.192387,0.079120,0.064520,0.226974,0.213509,-0.100083
Alojamiento,0.386291,1.000000,0.453010,0.134222,0.271922,0.197934,0.448568,0.422288,0.269433
Cuidado de la salud y el medio ambiente,0.213303,0.453010,1.000000,0.304659,0.470317,0.490234,0.865796,0.325397,0.069271
Crimen,0.192387,0.134222,0.304659,1.000000,0.286578,0.074416,0.389483,0.344627,0.259995
Transporte,0.079120,0.271922,0.470317,0.286578,1.000000,0.335992,0.464806,0.364716,0.059247
Educación,0.064520,0.197934,0.490234,0.074416,0.335992,1.000000,0.373279,0.077807,0.119700
Artes,0.226974,0.448568,0.865796,0.389483,0.464806,0.373279,1.000000,0.378659,0.075673
Recreación,0.213509,0.422288,0.325397,0.344627,0.364716,0.077807,0.378659,1.000000,0.173531
Economía,-0.100083,0.269433,0.069271,0.259995,0.059247,0.119700,0.075673,0.173531,1.000000


In [20]:
# Número de gráficas de dispersión bivariadas
num_variables = df.shape[1]
num_scatter_plots = math.comb(num_variables, 2)
print(
    f"Se deberían examinar {num_scatter_plots} "
    f"gráficas de dispersión bivariadas."
)

Se deberían examinar 36 gráficas de dispersión bivariadas.


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

### 3.  Cálculo y selección de componentes principales.

Utilizando Álgebra Lineal, calcule los eigen valores y los eigen vectores, a partir de ellos obtenga los Componentes Principales. Tenga en cuenta de estandarizar las variables y explicar por qué es importante no omitir este paso. 

Luego, examine los valores propios para determinar cuántos componentes principales se deben considerar. Explique el criterio que utilizó para la elección y justifique su respuesta.

In [26]:
# Utilice este espacio para escribir el código.
# Estandarización de las variables
scaler = StandardScaler()
df_standardized = scaler.fit_transform(df)

# Cálculo de la matriz de covarianza
cov_matrix = np.cov(df_standardized, rowvar=False)

# Cálculo de eigenvalores y eigenvectores
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Ordenar los eigenvalores y eigenvectores en orden descendente
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Mostrar los eigenvalores
print("Eigenvalores:")
print(eigenvalues)

# Mostrar los eigenvectores
print("\nEigenvectores:")
print(eigenvectors)

# Determinar el número de componentes principales
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Mostrar la proporción de varianza explicada acumulada
print("\nProporción acumulada de varianza explicada:")
print(cumulative_variance_ratio)

# Selección de componentes principales
num_components = np.argmax(cumulative_variance_ratio >= 0.8) + 1
print(f"\nNúmero de componentes principales seleccionados: {num_components}")

Eigenvalores:
[3.418683 1.217677 1.144959 0.923725 0.755581 0.632484 0.494551 0.319008
 0.120769]

Eigenvectores:
[[ 2.064140e-01  2.178353e-01  6.899560e-01  1.373212e-01 -3.691499e-01
  -3.746047e-01  8.470577e-02  3.623083e-01  1.391351e-03]
 [ 3.565216e-01  2.506240e-01  2.081722e-01  5.118287e-01  2.334878e-01
   1.416398e-01  2.306386e-01 -6.138551e-01  1.360034e-02]
 [ 4.602146e-01 -2.994653e-01  7.324926e-03  1.470183e-02 -1.032405e-01
   3.738480e-01 -1.386761e-02  1.856761e-01 -7.163549e-01]
 [ 2.812984e-01  3.553423e-01 -1.851050e-01 -5.390505e-01 -5.239397e-01
  -8.092329e-02 -1.860646e-02 -4.300248e-01 -5.860846e-02]
 [ 3.511508e-01 -1.796045e-01 -1.463763e-01 -3.029037e-01  4.043485e-01
  -4.675918e-01  5.833910e-01  9.359866e-02  3.629453e-03]
 [ 2.752926e-01 -4.833821e-01 -2.297025e-01  3.354110e-01 -2.088191e-01
  -5.021698e-01 -4.261819e-01 -1.886676e-01  1.108402e-01]
 [ 4.630545e-01 -1.947899e-01  2.648430e-02 -1.010804e-01 -1.050976e-01
   4.618807e-01  2.152515e-0

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones).

### 4.  Interpretación de los componentes principales. 

Logró reducir la dimensión en el ejercicio anterior? Si lo hizo, pueden los pesos obtenidos identificar dimensiones menores de los datos? Provea una interpretación a estas dimensiones, para ello puede utilizar gráficas y/o tablas.

In [1]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones).

#### Interpretación de los Componentes Principales
En el ejercicio anterior, logramos reducir la dimensión de las variables originales de 9 a 5 componentes principales, utilizando el criterio de proporción acumulada de varianza explicada (80%). Esto significa que los 5 componentes seleccionados explican el 82.64% de la variabilidad total de los datos, lo cual es suficiente para representar la información de manera adecuada.

Los pesos obtenidos (eigenvectores) permiten identificar las dimensiones menores de los datos. Cada componente principal es una combinación lineal de las variables originales, y los coeficientes asociados a cada variable indican su contribución a dicho componente. Esto facilita la interpretación de las dimensiones reducidas.

##### Interpretación de las Dimensiones
1. **Primer Componente Principal (PC1)**: Representa una combinación de variables como "Artes", "Cuidado de la salud y el medio ambiente", y "Alojamiento". Este componente podría interpretarse como una dimensión relacionada con la calidad cultural y ambiental de las ciudades.
2. **Segundo Componente Principal (PC2)**: Está influenciado por variables como "Recreación" y "Educación". Este componente podría interpretarse como una dimensión relacionada con las oportunidades recreativas y educativas.
3. **Tercer Componente Principal (PC3)**: Tiene una alta contribución de "Clima y Terreno" y "Crimen". Este componente podría interpretarse como una dimensión que refleja las condiciones climáticas y de seguridad.
4. **Cuarto Componente Principal (PC4)**: Está asociado con "Transporte" y "Economía". Este componente podría interpretarse como una dimensión relacionada con la infraestructura y el desarrollo económico.
5. **Quinto Componente Principal (PC5)**: Representa una combinación de variables con menor influencia, pero podría estar relacionado con aspectos específicos de las ciudades.

##### Visualización
Para facilitar la interpretación, se pueden generar gráficos de barras que muestren los pesos de las variables en cada componente principal, así como gráficos de dispersión que representen las ciudades en el espacio reducido de los componentes principales.

##### Conclusión
La reducción de dimensiones permite simplificar el análisis de los datos, identificando patrones y relaciones entre las ciudades de manera más clara. Las dimensiones obtenidas ofrecen una perspectiva más manejable para interpretar la calidad de vida en las ciudades, destacando los aspectos más relevantes de cada una.

### 5.   Detección de ciudades inusuales.

A partir de los Componentes Principales identifique las 5 ciudades calidad de vida inusualmente alta. En la distribución de calidad de vida  de la muestra que tan inusuales son? Cuáles son las dimensiones que hacen que estas ciudades tengan esta calidad de vida inusual?

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones).

In [31]:
# Proyección de los datos en el espacio de los componentes principales
principal_components = df_standardized @ eigenvectors[:, :num_components]

# Calcular la puntuación total de calidad de vida como la suma ponderada de los componentes principales
weights = explained_variance_ratio[:num_components]
scores = principal_components @ weights

# Identificar las 5 ciudades con las puntuaciones más altas
df_scores = pd.DataFrame(scores, index=df.index, columns=["Score"])
top_cities = df_scores.nlargest(5, "Score")

print("Ciudades con calidad de vida inusualmente alta:")
print(top_cities)

# Obtener las posiciones numéricas de las ciudades seleccionadas
indices = [df.index.get_loc(city) for city in top_cities.index]

# Seleccionar los componentes principales correspondientes
top_cities_components = principal_components[indices, :]

# Crear el DataFrame con los componentes principales de las ciudades seleccionadas
df_top_cities_components = pd.DataFrame(
    top_cities_components,
    index=top_cities.index,
    columns=[f"PC{i+1}" for i in range(num_components)]
)

print("\nContribución de los componentes principales para las ciudades seleccionadas:")
print(df_top_cities_components)

Ciudades con calidad de vida inusualmente alta:
                              Score
Ciudad                             
New-York,NY                3.968961
San-Francisco,CA           3.288833
Los-Angeles,Long-Beach,CA  2.881151
Boston,MA                  2.140208
Stamford,CT                2.139382

Contribución de los componentes principales para las ciudades seleccionadas:
                                 PC1       PC2       PC3       PC4       PC5
Ciudad                                                                      
New-York,NY                12.445179 -2.064887  0.210083 -3.430722 -1.685713
San-Francisco,CA            7.402662  1.228869  1.694329  0.406496  0.755162
Los-Angeles,Long-Beach,CA   7.257344  0.477878  2.054085 -0.522492 -1.657330
Boston,MA                   6.310655 -1.611192 -0.363214  0.276460 -0.173549
Stamford,CT                 3.451349  1.089921  1.659924  4.247819  0.479805
